In [2]:
import pandas as pd
pd.options.display.max_columns = None



In [3]:
df = pd.read_csv("../../../files/df_nulos_imputados.csv", index_col=0)

In [4]:
df.head()

,EmployeeID,Age,Gender,MaritalStatus,DateBirth,Education,EducationField,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Attrition,OverTime,DistanceFromHome,RemoteWork,BusinessTravel,TrainingTimesLastYear,JobRole,JobLevel,DailyRate,HourlyRate,MonthlyRate,PercentSalaryHike,StockOptionLevel,Salary
0,162,51,Male,Unknown,1972,3,Unknown,7,11.144344,20,15,15,1,3,3,3.000000,3,3.0,No,0.000000,6,1,Unknown,5,research director,5,684.0,51.0,6462.0,13,0,1000000000
1,259,52,Male,Unknown,1971,4,Life Sciences,0,34.000000,33,11,9,1,3,1,3.000000,2,3.0,No,0.265372,1,1,Unknown,5,manager,5,699.0,65.0,5678.0,14,1,1000000000
2,319,42,Male,Married,1981,2,Technical Degree,1,22.000000,22,11,15,1,4,4,2.787874,3,3.0,No,0.000000,4,1,travel_rarely,3,manager,5,532.0,58.0,4933.0,11,0,1000000000
3,nan1,47,Female,Married,1976,4,Medical,3,10.844380,20,5,6,1,3,2,2.793950,3,3.0,No,0.257984,2,0,travel_rarely,2,research director,4,359.0,82.0,26703.0,19,2,1000000000
4,nan2,46,Female,Divorced,1977,3,Technical Degree,2,11.732127,19,2,8,1,1,4,3.000000,4,3.0,No,0.000000,3,0,Unknown,5,sales executive,4,1319.0,45.0,7739.0,12,1,1000000000


In [5]:
df['OverallSatisfaction'] = df.apply(lambda x: (((x['EnvironmentSatisfaction'] + x['JobSatisfaction'] + x['RelationshipSatisfaction']) / 3)), axis=1)

In [6]:
df['OverallSatisfaction'].isna().sum()

0

In [7]:
df['OverallSatisfaction'].unique()

array([2.33333333, 1.66666667, 3.        , 2.        , 1.33333333,
       2.66666667, 1.        , 4.        , 3.33333333, 3.66666667])

In [8]:
satisfaction_scale = {1. : 1, 1.33333333 : 1, 2: 1.66666667, 2: 2. ,  2.33333333: 2,  2.66666667: 3, 3.: 3, 3.33333333: 3, 3.66666667: 4, 4.: 5}

In [9]:
def cambio_a_5(dato):
    return int(round(dato * 1.25))
        


In [10]:
df['OverallSatisfaction'] = df['OverallSatisfaction'].apply(cambio_a_5)

In [11]:
df['OverallSatisfaction'].unique()

array([3, 2, 4, 1, 5])

In [12]:
control = df[df['OverallSatisfaction'] >= 3]
variante = df[df['OverallSatisfaction'] < 3]

In [13]:
control

,EmployeeID,Age,Gender,MaritalStatus,DateBirth,Education,EducationField,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Attrition,OverTime,DistanceFromHome,RemoteWork,BusinessTravel,TrainingTimesLastYear,JobRole,JobLevel,DailyRate,HourlyRate,MonthlyRate,PercentSalaryHike,StockOptionLevel,Salary,OverallSatisfaction
0,162,51,Male,Unknown,1972,3,Unknown,7,11.144344,20,15,15,1,3,3,3.000000,3,3.0,No,0.000000,6,1,Unknown,5,research director,5,684.0,51.000000,6462.0,13,0,1000000000,3
2,319,42,Male,Married,1981,2,Technical Degree,1,22.000000,22,11,15,1,4,4,2.787874,3,3.0,No,0.000000,4,1,travel_rarely,3,manager,5,532.0,58.000000,4933.0,11,0,1000000000,4
5,1900,48,Female,Unknown,1975,3,Medical,3,10.619556,22,4,7,1,4,2,3.000000,3,3.0,No,0.000000,22,1,Unknown,3,manager,4,117.0,58.000000,2437.0,11,1,1000000000,3
7,387,42,Male,Married,1981,1,Unknown,1,20.000000,20,11,6,1,3,3,3.000000,3,4.0,No,0.000000,1,0,travel_rarely,3,sales executive,2,635.0,99.000000,24532.0,25,0,1000000000,3
13,58,41,Female,Married,1982,3,Unknown,1,11.770215,22,15,8,1,3,4,3.000000,3,3.0,Yes,0.000000,12,0,travel_rarely,0,research director,5,1360.0,49.000000,16280.0,12,0,1000000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,842,55,Male,Single,1968,1,Medical,4,10.914278,9,7,3,1,4,3,2.000000,3,3.0,Yes,0.279884,2,1,Unknown,3,sales executive,2,436.0,37.000000,21519.0,16,0,1000000000,3
1553,nan409,28,Female,Unknown,1995,4,Unknown,1,8.000000,8,7,5,4,2,1,3.000000,3,4.0,No,0.268735,43,0,Unknown,2,sales executive,2,760.0,81.000000,3698.0,20,0,1000000000,3
1555,927,41,Female,Divorced,1982,4,Life Sciences,4,11.516117,7,1,0,1,4,3,2.000000,1,3.0,Yes,1.000000,2,1,Unknown,1,laboratory technician,1,1085.0,66.199761,17725.0,13,1,1000000000,3
1558,nan101,31,Female,Unknown,1992,3,Medical,0,10.000000,9,8,5,1,4,3,1.000000,3,3.0,No,0.282617,12,1,Unknown,2,healthcare representative,2,616.0,41.000000,17369.0,11,2,1000000000,3


In [14]:
sin_rotacion = (df[df['Attrition'] == 'No'].shape[0] / df.shape[0]) * 100

In [15]:
sin_rotacion

83.87715930902111

In [16]:
con_rotacion = (df[df['Attrition'] == 'Yes'].shape[0] / df.shape[0]) * 100

In [17]:
con_rotacion

16.122840690978887

In [19]:
#sin_rotacion_control = df['Attrition'] == 'No' & control # aquí se nos ha hecho bola 

In [24]:
# Hecho en dos pasos, subgrupo con los Yes y luego se pasa al grupo de control
Attrition_positiva = df['Attrition'] == "Yes"

sin_rotacion_control = control[Attrition_positiva]

sin_rotacion_control

/var/folders/8b/dscb3hf534s_7dk8087c1m9m0000gn/T/ipykernel_89060/4049714752.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sin_rotacion_control = control[Attrition_positiva]


,EmployeeID,Age,Gender,MaritalStatus,DateBirth,Education,EducationField,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Attrition,OverTime,DistanceFromHome,RemoteWork,BusinessTravel,TrainingTimesLastYear,JobRole,JobLevel,DailyRate,HourlyRate,MonthlyRate,PercentSalaryHike,StockOptionLevel,Salary,OverallSatisfaction
13,58,41,Female,Married,1982,3,Unknown,1,11.770215,22,15,8,1,3,4,3.000000,3,3.0,Yes,0.000000,12,0,travel_rarely,0,research director,5,1360.0,49.000000,16280.0,12,0,1000000000,3
16,825,58,Male,Single,1965,4,Unknown,7,40.000000,31,13,8,1,2,4,3.000000,3,3.0,Yes,1.000000,2,0,travel_rarely,2,research director,5,286.0,31.000000,25761.0,12,0,1000000000,3
26,nan13,52,Female,Married,1971,1,Unknown,1,33.000000,32,6,9,1,4,4,3.000000,1,3.0,Yes,0.000000,2,1,Unknown,3,manager,5,266.0,57.000000,25846.0,15,1,1000000000,4
81,nan26,58,Female,Married,1965,4,Medical,1,40.000000,40,15,6,1,4,4,2.000000,3,3.0,Yes,0.237531,23,1,travel_rarely,3,healthcare representative,3,147.0,94.000000,3465.0,12,1,1000000000,4
132,780,33,Male,Single,1990,4,Other,1,10.000000,10,7,8,1,4,3,2.000000,3,3.0,Yes,0.270691,1,0,travel_rarely,2,research scientist,1,527.0,63.000000,5207.0,13,0,1000000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,376,47,Male,Married,1976,4,Life Sciences,1,11.128832,10,9,9,1,2,4,2.000000,3,3.0,Yes,0.262345,29,0,non-travel,2,manager,3,666.0,88.000000,10268.0,12,1,1000000000,3
1518,401,26,Male,Single,1997,4,Unknown,1,11.390549,8,7,4,1,4,2,2.756685,2,3.0,Yes,0.286703,4,1,travel_rarely,0,sales executive,2,950.0,48.000000,8450.0,12,0,1000000000,3
1544,nan406,28,Female,Unknown,1995,2,Unknown,7,10.000000,7,3,7,1,2,4,3.000000,1,3.0,Yes,1.000000,1,1,Unknown,4,research scientist,1,654.0,67.000000,3872.0,13,0,1000000000,3
1552,842,55,Male,Single,1968,1,Medical,4,10.914278,9,7,3,1,4,3,2.000000,3,3.0,Yes,0.279884,2,1,Unknown,3,sales executive,2,436.0,37.000000,21519.0,16,0,1000000000,3
